# Load the datasets

In [2]:
import pandas as pd
import numpy as np

In [3]:
x_train_final=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_final.csv")
x_test_final=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_final.csv")

In [4]:
print(f"Train : {x_train_final.shape}")
print(f"Test : {x_test_final.shape}")

Train : (246008, 9)
Test : (61503, 9)


In [5]:
#load y_train and y_test
y_train=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_train.csv")
y_test=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_test.csv")

In [6]:
print(f"Train : {y_train.shape}")
print(f"Test : {y_test.shape}")

Train : (246008, 1)
Test : (61503, 1)


# FIT THE LOGIT MODEL

In [7]:
import statsmodels.api as sm

In [8]:
#ADD THE CONSTANT
x=sm.add_constant(x_train_final)

In [9]:
#fit the model
model=sm.Logit(y_train,x).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.252557
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 TARGET   No. Observations:               246008
Model:                          Logit   Df Residuals:                   245998
Method:                           MLE   Df Model:                            9
Date:                Mon, 23 Feb 2026   Pseudo R-squ.:                 0.09977
Time:                        23:41:23   Log-Likelihood:                -62131.
converged:                       True   LL-Null:                       -69017.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -2.4354      0.008   -299.825      0.000      -2.451      -2.419
EXT_SOURCE_3           -0.8476      0.014    -60.300      0.000      -0.875      -0.820
EXT_SOURCE_2           -0.8168      0.014    -57.812      0.000      -0.845      -0.789
EXT_SOURCE_1           -0.5306      0.022    -23.621      0.000      -0.575      -0.487
DAYS_EMPLOYED          -0.3423      0.027    -12.469      0.000      -0.396      -0.288
AMT_GOODS_PRICE        -0.6506      0.025    -25.643      0.000      -0.700      -0.601
OCCUPATION_TYPE        -0.3236      0.030    -10.944      0.000      -0.382      -0.266
ORGANIZATION_TYPE      -0.4154      0.037    -11.197      0.000      -0.488      -0.343
NAME_INCOME_TYPE       -0.0896      0.041     -2.186      0.029      -0.170      -0.009
NAME_EDUCATION_TYPE    -0.6236      0.037    -17.035      0.000      -0.695      -0.552
=======================================================================================
"""

# PREDICT PROBABILITIES

In [10]:
x_train_final=sm.add_constant(x_train_final)
x_test_final=sm.add_constant(x_test_final)

In [11]:
y_prob_train=model.predict(x_train_final)
y_prob_test=model.predict(x_test_final)

# CALCULATE AUC, SOMER'S D OF TRAIN AND TEST

In [12]:
from sklearn.metrics import roc_auc_score

In [13]:
auc_train=roc_auc_score(y_train,y_prob_train)
auc_test=roc_auc_score(y_test,y_prob_test)

In [14]:
print(f"Auc train : {auc_train}")
print(f"Auc test : {auc_test}")

Auc train : 0.7370846175274252
Auc test : 0.7318805945627316


In [15]:
# CALCULATE SOMER'S D OF TEST AND TRAIN
somers_d_train=2*auc_train-1
somers_d_test=2*auc_test-1

In [16]:
print(f"Somer's D train : {somers_d_train}")
print(f"Somer's D test : {somers_d_test}")

Somer's D train : 0.4741692350548503
Somer's D test : 0.46376118912546316


# CALCULATE KS-STATISTIC

In [17]:
y_prob_train = pd.Series(y_prob_train, name="pd")

In [18]:
y_train = y_train.squeeze()

In [19]:
ks_train=pd.DataFrame({"y_train":y_train,"pd":y_prob_train})

In [20]:
print("y_train shape:", y_train.shape)
print("y_prob_train shape:", y_prob_train.shape)

y_train shape: (246008,)
y_prob_train shape: (246008,)


In [21]:
ks_train.sort_values("pd",ascending=False,inplace=True)

In [22]:
ks_train['decile']=pd.qcut(ks_train['pd'],q=10)

In [23]:
ks_train

,y_train,pd,decile
236377,0,0.611063,"(0.173, 0.611]"
90480,0,0.601054,"(0.173, 0.611]"
180558,1,0.573671,"(0.173, 0.611]"
144475,0,0.569529,"(0.173, 0.611]"
146710,1,0.567775,"(0.173, 0.611]"
...,...,...,...
109034,0,0.002505,"(0.0005399999999999999, 0.0191]"
102145,0,0.002295,"(0.0005399999999999999, 0.0191]"
229372,0,0.001845,"(0.0005399999999999999, 0.0191]"
84074,0,0.001651,"(0.0005399999999999999, 0.0191]"


In [24]:
ks_train=ks_train.groupby('decile',as_index=False).agg(
    total=('y_train', 'count'),
    bad=('y_train', 'sum'))

C:\Users\owner's\AppData\Local\Temp\ipykernel_17828\285423254.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ks_train=ks_train.groupby('decile',as_index=False).agg(


In [25]:
ks_train['good']=ks_train['total']-ks_train['bad']

In [26]:
ks_train=ks_train.sort_values("decile",ascending=False)
ks_train

,decile,total,bad,good
9,"(0.173, 0.611]",24601,6100,18501
8,"(0.122, 0.173]",24601,3607,20994
7,"(0.093, 0.122]",24601,2592,22009
6,"(0.0735, 0.093]",24600,2027,22573
5,"(0.0586, 0.0735]",24601,1557,23044
4,"(0.0466, 0.0586]",24601,1265,23336
3,"(0.0368, 0.0466]",24600,964,23636
2,"(0.0277, 0.0368]",24553,796,23757
1,"(0.0191, 0.0277]",24589,557,24032
0,"(0.0005399999999999999, 0.0191]",24661,395,24266


In [27]:
ks_train['bad_cum_sum%']=ks_train['bad'].cumsum()/ks_train['bad'].sum()
ks_train['good_cum_sum%']=ks_train['good'].cumsum()/ks_train['good'].sum()

In [28]:
ks_train['ks']=ks_train['bad_cum_sum%']-ks_train['good_cum_sum%']

In [29]:
ks_stat=ks_train['ks'].max()

In [30]:
ks_stat

0.349570787899042

# CREATE A FUNCTION FOR KS STAT CALCULATION

In [31]:
def ks_stat_calculation(y,probability):
    y=y.squeeze()
    probability=pd.Series(probability).squeeze()
    df=pd.DataFrame({"y":y,"PD":probability})
    df.sort_values("PD",ascending=False,inplace=True)
    df['DECILE']=pd.qcut(df['PD'],q=10)
    df= df.groupby("DECILE",as_index=False).agg(total=('y','count'),bad=('y','sum'))
    df['good']=df['total']-df['bad']
    df['cum_sum_bad%']=df['bad'].cumsum()/df['bad'].sum()
    df['cum_sum_good%']=df['good'].cumsum()/df['good'].sum()
    df['ks']=abs(df['cum_sum_bad%']-df['cum_sum_good%'])
    ks_stat=df['ks'].max()
    return ks_stat

In [32]:
#test the function on y_train
print(f"KS TRAIN: {ks_stat_calculation(y_train,y_prob_train)}")

KS TRAIN: 0.34957078789904195


C:\Users\owner's\AppData\Local\Temp\ipykernel_17828\503547970.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df= df.groupby("DECILE",as_index=False).agg(total=('y','count'),bad=('y','sum'))


In [33]:
print(f"KS TEST: {ks_stat_calculation(y_test,y_prob_test)}")

C:\Users\owner's\AppData\Local\Temp\ipykernel_17828\503547970.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df= df.groupby("DECILE",as_index=False).agg(total=('y','count'),bad=('y','sum'))


KS TEST: 0.3396040243072622


# Inferences from Model Validation

|AUC|SOMERS D|KS STATS|STATUS|
|---|---|---|---|
|"0.75+"|"0.5+"|"35%"|"STRONG"|
|"0.7-0.75"|"0.4-0.5"|"30-35%"|"GOOD"|

1) AUC TRAIN ~ 0.737
2) AUC TEST ~ 0.731
3) SOMER'S D TRAIN~ 0.474
4) SOMER'S TEST~0.463
5) KS STAT TRAIN ~ 0.34
6) KS STATS TEST ~ 0.33

Train and test metrics are closely aligned, indicating good generalization and no evidence of overfitting.

# EXPORT MODEL COEFFICIENTS, MODEL SUMMARY, PREDICTED PD(TRAIN AND TEST) 

In [35]:
model.params

const                 -2.435404
EXT_SOURCE_3          -0.847634
EXT_SOURCE_2          -0.816845
EXT_SOURCE_1          -0.530618
DAYS_EMPLOYED         -0.342289
AMT_GOODS_PRICE       -0.650568
OCCUPATION_TYPE       -0.323617
ORGANIZATION_TYPE     -0.415435
NAME_INCOME_TYPE      -0.089649
NAME_EDUCATION_TYPE   -0.623591
dtype: float64

In [37]:
# Export coeeficients
model.params.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\final_model_coefficients.csv")

In [45]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 TARGET   No. Observations:               246008
Model:                          Logit   Df Residuals:                   245998
Method:                           MLE   Df Model:                            9
Date:                Mon, 23 Feb 2026   Pseudo R-squ.:                 0.09977
Time:                        23:47:19   Log-Likelihood:                -62131.
converged:                       True   LL-Null:                       -69017.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -2.4354      0.008   -299.825      0.000      -2.451      -2.419
EXT_SOURCE_3           -0.8476      0.014    -60.300      0.000      -0.875      -0.820
EXT_SOURCE_2           -0.8168      0.014    -57.812      0.000      -0.845      -0.789
EXT_SOURCE_1           -0.5306      0.022    -23.621      0.000      -0.575      -0.487
DAYS_EMPLOYED          -0.3423      0.027    -12.469      0.000      -0.396      -0.288
AMT_GOODS_PRICE        -0.6506      0.025    -25.643      0.000      -0.700      -0.601
OCCUPATION_TYPE        -0.3236      0.030    -10.944      0.000      -0.382      -0.266
ORGANIZATION_TYPE      -0.4154      0.037    -11.197      0.000      -0.488      -0.343
NAME_INCOME_TYPE       -0.0896      0.041     -2.186      0.029      -0.170      -0.009
NAME_EDUCATION_TYPE    -0.6236      0.037    -17.035      0.000      -0.695      -0.552
=======================================================================================
"""

In [47]:
summary_table = pd.DataFrame({
    "Variable": model.params.index,
    "Coefficient": model.params.values,
    "Std_Error": model.bse.values,
    "z_value": model.tvalues.values,
    "p_value": model.pvalues.values
})

In [48]:
summary_table.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\model_summary.csv")

In [46]:
#Export predicted probabilities from train and test
y_prob_test.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\test_pd.csv")
y_prob_train.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\train_pd.csv")

## Author
Pranjal Saxena